In [ ]:
import numpy as np
import nltk
import string
import random
import warnings
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
 nltk.download('punkt')
nltk.download('wordnet')
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# Step 1: Read the CSV file
df = pd.read_csv(r'/content/dataset.csv')


In [ ]:
# Step 2: Preprocess the data
def preprocess_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalnum()]
    lemmer = nltk.stem.WordNetLemmatizer()
    tokens = [lemmer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

# Assuming 'symptoms' and 'disease' columns exist in your CSV file
df['preprocessed_symptoms'] = df['symptoms'].apply(preprocess_text)

In [ ]:
# Step 3: Train the machine learning model
tfidf_vectorizer = TfidfVectorizer()
symptoms_tfidf = tfidf_vectorizer.fit_transform(df['preprocessed_symptoms'])

In [ ]:
# Step 4: Implement the AI response system
def predict_disease(symptoms):
    preprocessed_input = preprocess_text(symptoms)
    input_tfidf = tfidf_vectorizer.transform([preprocessed_input])
    similarity_scores = cosine_similarity(input_tfidf, symptoms_tfidf)
    ranked_indices = np.argsort(similarity_scores)[0][::-1]  # Sort indices in descending order of similarity
    top_diseases = {}
    for idx in ranked_indices:
        disease = df['disease'].iloc[idx]
        if len(top_diseases) < 3:
            if disease not in top_diseases:
                top_diseases[disease] = similarity_scores[0][idx]
        else:
            break
    return top_diseases

In [ ]:
# Initialize the chatbot
def chat():
    print("Hello! How can I assist you today?")
    print("1. Book an appointment")
    print("2. Ask the AI about your symptoms")
    print("3. Provide feedback")
    print("4. Exit")
    while True:
        user_response = input("Please enter your choice: ")
        if user_response == '1':
            print("You have chosen to book an appointment. Please visit our website to book.")
        elif user_response == '2':
            symptoms = input("Please describe your symptoms: ")
            top_diseases = predict_disease(symptoms)
            print("Top diseases based on your symptoms:")
            for disease, score in top_diseases.items():
                cures = ', '.join(df[df['disease'] == disease]['cures'].values)
                print(f"{disease} (Similarity Score: {score})")
                print(f"   Cures: {cures}")
        elif user_response == '3':
            feedback = input("Please provide your feedback: ")
            print("Thank you for your feedback! We will use it to improve our services.")
        elif user_response == '4':
            print("Goodbye!")
            break
        else:
            print("Invalid choice. Please enter a number from 1 to 4.")


In [ ]:
# Main function to initiate the chat
if __name__ == "__main__":
    chat()

Hello! How can I assist you today?
1. Book an appointment
2. Ask the AI about your symptoms
3. Provide feedback
4. Exit
You have chosen to book an appointment. Please visit our website to book.
Invalid choice. Please enter a number from 1 to 4.
Invalid choice. Please enter a number from 1 to 4.
Invalid choice. Please enter a number from 1 to 4.
